# Evaluation

In [17]:
import os, logging
import json
import subprocess
from pathlib import Path
import fnmatch

## Setup

### Persist Results

In [18]:
storedir = None  # Set this to persist evaluation results

In [19]:
if storedir is not None:
    data_storedir = f"{storedir}/data"
    Path(data_storedir).mkdir(exist_ok=True)
else:
    checkpoint_storedir = None
    data_storedir = None
    
try:
    job_id = os.environ['PBS_JOBID'].split('.pbs')[0]
except KeyError:
    job_id = 'local'

In [20]:
logger = logging.getLogger('job')

### Imports

In [21]:
logger.info('Importing third-party packages ...')

import torch

from op_ds.gno.gno import GNOLayer, GNO
from op_ds.gno.kernel import NonlinearKernelTransformWithSkip
from op_ds.utils.fnn import FNN
from volatility_smoothing.utils.train import Trainer
from volatility_smoothing.utils.data import OptionsDataset

### Device

In [22]:
logger.info(f"Defining device (torch.cuda.is_available()={torch.cuda.is_available()})")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logger.info(f'Running using device `{device}`')

if device.type == 'cuda':
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    formatted_result = str(result.stdout).replace('\\n', '\n').replace('\\t', '\t')##

    logger.info(formatted_result)
    logger.info(f'Device count: {torch.cuda.device_count()}')
    logger.info(f'Visible devices count: {os.environ["CUDA_VISIBLE_DEVICES"]}')

## Dataset

In [82]:
data_dir = "../data"

with open(f"{data_dir}/fields.json", 'r') as f:
    fields = json.load(f)

mapping = {
    'option_type': fields.index('option_type'),
    'time_to_maturity': fields.index('time_to_maturity'),
    'log_moneyness': fields.index('log_moneyness'),
    'implied_volatility': fields.index('implied_volatility_lbr'),
    'bid': fields.index('bid'),
    'ask': fields.index('ask'),
    'discount_factor': fields.index('discount_factor'),
    'underlying_forward': fields.index('underlying_forward')
}

dev_dir = f"{data_dir}/dev"


def read_filepaths(dir):
    return [f"{dir}/{filename}" for filename in fnmatch.filter(os.listdir(dir), '*.pt')]


dev_dataset = OptionsDataset(read_filepaths(dev_dir), mapping, subsample=False)

## Model and Optimizer

### Instantiate

In [83]:
in_channels = 1
out_channels = 1
channels = (in_channels, 16, 16, 16, out_channels)
spatial_dim = 2
gno_channels = 16
hidden_channels = 64

gno_layers = []

for i in range(m := (len(channels) - 1)):
    lifting = FNN.from_config((channels[i], hidden_channels, gno_channels), hidden_activation='gelu', batch_norm=False)
    projection = None if i < m - 1 else FNN.from_config((gno_channels, hidden_channels, channels[i+1]), hidden_activation='gelu', batch_norm=False)
    transform = NonlinearKernelTransformWithSkip(in_channels=gno_channels, out_channels=gno_channels, skip_channels=in_channels, spatial_dim=spatial_dim, hidden_channels=(hidden_channels, hidden_channels), hidden_activation='gelu', batch_norm=False)

    if i == 0:
        local_linear = False
    else:
        local_linear = True
        
    activation = torch.nn.GELU() if i < m - 1 else torch.nn.Softplus(beta=0.5)
        
    gno_layer = GNOLayer(gno_channels, transform=transform, local_linear=local_linear, local_bias=True,
                         activation=activation, lifting=lifting, projection=projection)
    gno_layers.append(gno_layer)
    
gno = GNO(*gno_layers, in_channels=in_channels).to(device)

In [84]:
optimizer = torch.optim.AdamW(gno.parameters())

### Load Checkpoint

In [85]:
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    logger.info(f"Loaded checkpoint from {path}")
    return model, optimizer

In [86]:
path = "../train/store/9448705/checkpoints/checkpoint_final.pt"
gno, optimizer = load_checkpoint(gno, optimizer, path)

## Evaluation

In [87]:
step_r = 0.05
step_z = 0.05
subsample_size = 35
training_io = Trainer(step_r=step_r, step_z=step_z)

In [88]:
df_val, df_rel, df_fit = training_io.evaluate(gno, dev_dataset, device=device, storedir=storedir, logger=logger, step_r=step_r, step_z=step_z)

In [90]:
df_fit

,mean,std,min,%05,%25,%50,%75,%95,max
quote_datetime,,,,,,,,,
2012-02-24 09:50:00,0.307599,0.374522,0.001840,0.030601,0.101431,0.189978,0.366928,0.911127,3.351730
2012-03-27 10:10:00,0.237303,0.264122,0.001482,0.014741,0.070680,0.153551,0.308814,0.731986,2.488052
2012-03-27 14:10:00,0.265702,0.306287,0.000155,0.010549,0.066785,0.172293,0.351501,0.775467,2.464123
2012-04-10 14:10:00,0.209695,0.232821,0.000409,0.008389,0.063442,0.143885,0.279517,0.674965,1.734750
2012-04-11 12:50:00,0.230648,0.241801,0.000039,0.013066,0.064616,0.142781,0.302760,0.703193,1.601363
2012-05-03 16:10:00,0.371304,0.713999,0.000173,0.011454,0.058635,0.167248,0.440043,1.335852,12.991895
2012-05-07 14:10:00,0.370161,0.750800,0.000538,0.014409,0.078295,0.197411,0.365035,1.155832,9.544945
2012-05-08 12:50:00,0.283818,0.394815,0.000103,0.015326,0.084501,0.176593,0.344320,0.837842,4.311754
